## Libraries

In [1]:
import ee
import sys
import geemap
import geemap.colormaps as cm

ee.Authenticate()
ee.Initialize()


from functools import partial


ACOLITE_PATH = r'C:\Users\sergi\Documents\repos\gee_acolite\src'

sys.path.append(ACOLITE_PATH)

import ACOLITE

## L2 from GEE with sen2cor

In [2]:
# def to_rrs(image):
#     return image.divide(10_000)

# a = []
# for (bounds, start, end, tile) in [                                
#                                 ([-60.827179, -63.048731, -60.420685, -62.867057], '2016-11-13', '2016-11-14', '21EUL'),
#                                 ([-60.827179, -63.048731, -60.420685, -62.867057], '2020-01-19', '2020-01-20', '21EUL'),
#                                 ([-60.827179, -63.048731, -60.420685, -62.867057], '2020-12-27', '2020-12-28', '21EUL'),
#                                 ([-60.827179, -63.048731, -60.420685, -62.867057], '2021-11-22', '2021-11-23', '21EUL'),
#                                 ([-60.827179, -63.048731, -60.420685, -62.867057], '2022-03-29', '2022-03-30', '21EUL'),
#                                 ([-60.827179, -63.048731, -60.420685, -62.867057], '2023-03-17', '2023-03-18', '21EUL'),
#                                 ([-60.827179, -63.048731, -60.420685, -62.867057], '2023-09-10', '2023-09-11', '21EUL'),
#                                 ([-60.827179, -63.048731, -60.420685, -62.867057], '2024-10-24', '2024-10-25', '21EUL'),
#                                 ([-60.827179, -63.048731, -60.420685, -62.867057], '2025-01-25', '2025-01-26', '21EUL'),
#                                 ]:
      
#     roi = ee.Geometry.Rectangle(bounds)
#     a.append( ACOLITE.search_with_clouds(roi, start, end, tile = tile, collection = 'S2_SR_HARMONIZED').map(to_rrs).first() )
     
# sentinel2_l2r_rrs = ee.ImageCollection.fromImages(a)

# print(f'Collection size {sentinel2_l2r_rrs.size().getInfo()}')

# sentinel2_l2w_rrs = sentinel2_l2r_rrs.map(ACOLITE.mask_water_by_SWIR1)
# for product in [ACOLITE.add_zgreen, ACOLITE.add_zred]:
#         sentinel2_l2w_rrs = sentinel2_l2w_rrs.map(product)

# sentinel2_l2w_rrs = ee.ImageCollection.fromImages([sentinel2_l2w_rrs.max()])

## GEE + ACOLITE integration

In [3]:
images = []
for (bounds, start, end, tile) in [                                
                                ([-60.827179, -63.048731, -60.420685, -62.867057], '2016-11-13', '2016-11-14', '21EUL'),
                                # ([-60.827179, -63.048731, -60.420685, -62.867057], '2020-01-19', '2020-01-20', '21EUL'),
                                # ([-60.827179, -63.048731, -60.420685, -62.867057], '2020-12-27', '2020-12-28', '21EUL'),
                                # ([-60.827179, -63.048731, -60.420685, -62.867057], '2021-11-22', '2021-11-23', '21EUL'),
                                # ([-60.827179, -63.048731, -60.420685, -62.867057], '2022-03-29', '2022-03-30', '21EUL'),
                                # ([-60.827179, -63.048731, -60.420685, -62.867057], '2023-03-17', '2023-03-18', '21EUL'),
                                # ([-60.827179, -63.048731, -60.420685, -62.867057], '2023-09-10', '2023-09-11', '21EUL'),
                                # ([-60.827179, -63.048731, -60.420685, -62.867057], '2024-10-24', '2024-10-25', '21EUL'),
                                # ([-60.827179, -63.048731, -60.420685, -62.867057], '2025-01-25', '2025-01-26', '21EUL'),
                                ]:
      
    roi = ee.Geometry.Rectangle(bounds)
    images.append( ACOLITE.search_with_clouds(roi, start, end, tile = tile).first() )
     
tolerance = 10
mask_clouds_by_probability = partial(ACOLITE.mask_clouds_by_probability, threshold = tolerance)

sentinel2_l1 = ee.ImageCollection.fromImages(images).map(mask_clouds_by_probability)
print(f'Collection size {sentinel2_l1.size().getInfo()}')

sentinel2_l1r_rrs = sentinel2_l1.map(ACOLITE.select_sentinel2_bands).map(ACOLITE.to_rrs) # Digital Number to rrs
sentinel2_list = sentinel2_l1r_rrs.toList(sentinel2_l1r_rrs.size()) # Collection to List for iteration
# pre processing

# during
images = [ ACOLITE.dask_spectrum_fitting(ee.Image(sentinel2_list.get(i)))[0] for i in range(sentinel2_l1r_rrs.size().getInfo()) ]
sentinel2_l2r_rrs = ee.ImageCollection.fromImages( images )
# during

# post processing
sentinel2_l2w_rrs = sentinel2_l2r_rrs.map(ACOLITE.mask_water_by_SWIR1)

for product in [ACOLITE.add_zgreen, ACOLITE.add_zred]:
    sentinel2_l2w_rrs = sentinel2_l2w_rrs.map(product)
# post processing

Collection size 1
sel_aot=np.float64(0.001), sel_lut='ACOLITE-LUT-202110-MOD1', sza=50.1587515672, vza=8.248614186813846, raa=88.6513636419923


## SDB models creation

In [5]:
deception_calibration = geemap.shp_to_ee(r"C:\Users\sergi\Documents\repos\bathypy\data\shapefiles\deception_calibration\deception_calibration.shp")
deception_land = geemap.shp_to_ee(r"C:\Users\sergi\Documents\repos\bathypy\data\shapefiles\deception\deception.shp")
mask = ee.Image.constant(1).clip(deception_land).mask().Not()

max_depth = 6

in_situ = ee.Image('projects/ee-sergiohercar1/assets/Deception/DCI_UTM20_ALL').rename('depth')
in_situ = in_situ.updateMask(in_situ.lte(0)).multiply(-1)
valid_depth = in_situ.lte(max_depth)
in_situ = in_situ.updateMask(valid_depth)

deception_image = ee.ImageCollection.fromImages([sentinel2_l2w_rrs.select(['zred', 'zgreen']).max()]).first()


model = ee.Image()
for pseudomodel in ['zred', 'zgreen']:
    image_to_calibration = ee.Image(1).addBands(deception_image.select(pseudomodel).rename(pseudomodel))
    image_to_calibration = image_to_calibration.addBands(in_situ)

    linear_regression = image_to_calibration.reduceRegion(
        reducer=ee.Reducer.robustLinearRegression(numX=2, numY=1),
        scale = 10,
        geometry = deception_calibration
    )

    coefficients = ee.Array(linear_regression.get("coefficients"))
    n = coefficients.get([0, 0])
    m = coefficients.get([1, 0])

    print(pseudomodel, f'y = {m.getInfo():.3f}x {n.getInfo():+.3f}')

    regression = ee.Image().expression('m * x + n', {'x' : deception_image.select(pseudomodel), 
                                                     'n' : n, 'm' : m}).rename(pseudomodel.replace('z', 'SDB '))

    model = model.addBands(regression)

model = model.addBands(ACOLITE.switching_model(model.select('SDB green'), model.select('SDB red')))
model = model.updateMask(valid_depth)

zred y = 26.481x -29.305
zgreen y = 64.192x -63.697


## Plots

In [7]:
depth_plot = {
    'palette': cm.palettes.viridis_r,
    'min': 0,
    'max': 6,
}

error_plot = {
    'palette': cm.palettes.RdGy,
    'min': -2,
    'max': 2,
}

error = in_situ.subtract(model)

Map = geemap.Map()
Map.addLayer(model.select('SDB red'), depth_plot, f"SDB Red")
# Map.addLayer(model.select('SDB green'), depth_plot, f"SDB Green")
# Map.addLayer(model.select('switching'), depth_plot, f"SDB switching")
# Map.addLayer(error.select('SDB red'), error_plot, f"SDB Red Error")
# Map.addLayer(error.select('SDB green'), error_plot, f"SDB Green Error")
Map.addLayer(in_situ, depth_plot, f"In situ")

Map.center_object(deception_land)
Map.add_colorbar(depth_plot, label="Depth (m)")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# bands = ['SDB red', 'SDB green', 'switching']
# folder =  r'C:\Users\sergi\Documents\cadiz\proyectos\GOYAS\Metadata_Extraction'
# geemap.download_ee_image(model.select(bands), rf"{folder}\GEE_ACOLITE_multi_image.tif", 
#                          roi, 'EPSG:32721', scale = 10, dtype = 'float32')

In [ ]:
# bands = ['SDB red', 'SDB green', 'switching']
# abs_image = error.select(bands).abs()

# mean_abs = abs_image.reduceRegion(
#     reducer=ee.Reducer.mean(),
#     scale=10,
#     maxPixels=1e8
# )

# median_abs = abs_image.reduceRegion(
#     reducer=ee.Reducer.median(),
#     scale=10,
#     maxPixels=1e8
# )
# std_abs = abs_image.reduceRegion(
#     reducer=ee.Reducer.stdDev(),
#     scale=10,
#     maxPixels=1e8
# )

# print('Mean', mean_abs.getInfo())
# print('Median', median_abs.getInfo())
# print('Std', std_abs.getInfo())